In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.image as image
import seaborn as sns
import pandas as pd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from libpysal.weights import Queen
from esda.moran import Moran, Moran_Local
import numpy as np
from matplotlib.colors import ListedColormap
from splot.esda import lisa_cluster
import numpy as np
from scipy.interpolate import UnivariateSpline
import pymc as pm


sns.set_style("whitegrid")

In [ ]:
final = gpd.read_file('data/map/final.json')
district = gpd.read_file('data/map/district.json')
country = gpd.read_file('data/map/vietnam.json')

In [ ]:
final = final.to_crs(epsg=32648)
district = district.to_crs(epsg=32648)
country.to_crs(epsg=32648)
final['centroid'] = final.geometry.centroid
centroids = final.copy()
centroids.geometry = centroids['centroid']

In [ ]:
final = final.to_crs(epsg=4326)
district = district.to_crs(epsg=4326)
country = country.to_crs(epsg=4326)
centroids = centroids.to_crs(epsg=4326)

In [ ]:
country['is_namdinh'] = country['VARNAME_1'] == 'NamDinh'

In [ ]:
# Set the background color for both the main and inset maps
background_color = 'white'
# Creating the main district and commune map
fig_district, ax_district = plt.subplots(figsize=(8, 8))
ax_district.set_facecolor(background_color)
# Plotting communes with sky blue fill and red edges
final.plot(ax=ax_district, color='skyblue', edgecolor='red', linewidth=0.25)
# Plotting district boundaries in black
district.plot(ax=ax_district, color='none', edgecolor='black', linewidth=0.5, alpha=0.7)
# Creating the inset map
inset_ax = inset_axes(ax_district, width="35%", height="35%", loc=4)  # Bottom right corner
inset_ax.patch.set_facecolor(background_color)
country.plot(ax=inset_ax, color=background_color, edgecolor='black', linewidth=0.5)
# Zoom in on 'is_namdinh' area
namdinh_bounds = country[country['is_namdinh']].total_bounds
padding = 5.0  # Padding around the 'is_namdinh' area
x_range = namdinh_bounds[2] - namdinh_bounds[0]
y_range = namdinh_bounds[3] - namdinh_bounds[1]
inset_ax.set_xlim(namdinh_bounds[0] - x_range * padding, namdinh_bounds[2] + x_range * padding)
inset_ax.set_ylim(namdinh_bounds[1] - y_range * padding, namdinh_bounds[3] + y_range * padding)

country[country['is_namdinh']].plot(ax=inset_ax, color='skyblue', edgecolor='black', linewidth=0.5)
# centroids.plot(ax=ax_district, marker='o', color='red', markersize=1)
inset_ax.set_xticks([])
inset_ax.set_yticks([])

for spine in ax_district.spines.values():
    spine.set_linewidth(1)  # Making the border lines solid
    spine.set_edgecolor('black')  # Setting the border color to black

for spine in inset_ax.spines.values():
    spine.set_visible(True)  

# Removing gridlines
ax_district.grid(False)
inset_ax.grid(False)

# Custom legend for the map with reduced font size
legend_elements = [
    Patch(facecolor='skyblue', edgecolor='red', label='Communes'),
    Line2D([0], [0], color='black', linewidth=0.5, linestyle='-', label='District boundaries')
]

# Adding the custom legend to the plot with a reduced font size
ax_district.legend(handles=legend_elements, loc='upper left', fontsize=8)
ax_district.ticklabel_format(style='plain', axis='y', useOffset=False)

arrow_img = image.imread('data/images/arrow.png')
# Adding the arrow to the bottom left
arrow_position = (0.08, 0.06)  # Adjust as needed
arrowbox = OffsetImage(arrow_img, zoom=0.05)
ab = AnnotationBbox(arrowbox, arrow_position, xycoords='axes fraction', frameon=False)
ax_district.add_artist(ab)

plt.show()

In [ ]:
cm = gpd.read_file('data/map/commune.json')
cm = cm.sort_values(by='OBJECTID', ascending=True)

In [ ]:
# Assuming 'final' is your GeoDataFrame

wq = Queen.from_dataframe(cm)
# wq.transform='r'

# Plot the matrix as a heatmap
plt.imshow(wq.full()[0], cmap='binary', interpolation='nearest')
plt.title('Queen Contiguity Matrix')
plt.colorbar(label='Contiguity')
plt.show()

In [ ]:
final_sorted = final.sort_values(by=['OBJECTID', 'year'], ascending=[True, True])

In [ ]:
final_sorted['POP_DENS'] = final_sorted['pop'] / (final_sorted['Shape_Area'] / 1000000)

In [ ]:
final_sorted['total_pop_year'] = final_sorted.groupby('year')['pop'].transform('sum')

In [ ]:
final_sorted['total_case_year'] = final_sorted.groupby('year')['observed'].transform('sum')

In [ ]:
notif_data =  final_sorted.groupby('year').agg(total_case_year=('observed', 'sum'), total_pop_year=('pop', 'sum')).reset_index()

In [ ]:
notif_data['notification_rate'] = (notif_data['total_case_year'] / notif_data['total_pop_year']) * 100000

In [ ]:
notif_data['year'] = notif_data['year'].astype(int)

In [ ]:
notif_data = notif_data.sort_values('year')

In [ ]:
spline = UnivariateSpline(notif_data['year'], notif_data['notification_rate'], s=1)
x_range = np.linspace(notif_data['year'].min(), notif_data['year'].max(), 1000)
y_smooth = spline(x_range)

In [ ]:
# To plot this, for example:
plt.figure(figsize=(7.2, 2.8))
plt.plot(notif_data['year'], notif_data['notification_rate'], marker='o', linestyle='-', color='b')
plt.xticks(np.arange(2013, 2023, 1))
plt.legend()
plt.show()

In [ ]:
sum(notif_data['total_case_year'])

In [ ]:
final_sorted['expected'] = final_sorted['observed'] * final_sorted['total_pop_year'] / final_sorted['pop']

In [ ]:
final_sorted['expected'] = final_sorted['pop'] * final_sorted['total_case_year']/ final_sorted['total_pop_year']

In [ ]:
final_sorted['smr'] = final_sorted['observed'] / final_sorted['expected'] 

In [ ]:
for year in final_sorted['year'].unique():  # Assuming there's a 'year' column
    # Filter data for the year
    yearly_data = final_sorted[final_sorted['year'] == year]
    
    # Calculate global Moran's I
    mi = Moran(yearly_data['smr'], wq)
    print(f"Global Moran's I for year {year}: {mi.I}, p-value: {mi.p_sim}")
    
    # Calculate local Moran's I
    local_mi = Moran_Local(yearly_data['smr'], wq)
    local_i = local_mi.Is
    

In [ ]:
final_sorted.drop(columns='centroid', inplace=True)

In [ ]:
# final_sorted.to_file('data/map/final_sorted.json', driver='GeoJSON')

In [ ]:
final_sorted['adjusted_observed'] = final_sorted['observed'].apply(lambda x: x + 0.001 if x == 0 else x)

In [ ]:
final_sorted['adjusted_smr'] = final_sorted['adjusted_observed'] / final_sorted['expected']

In [ ]:
years = sorted(final_sorted['year'].unique())
num_years = len(years)
cols = 2
rows = (num_years + cols - 1) // cols  # Ceiling division

f, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(7.2, 2.8 * rows), constrained_layout=True)
axs = axs.flatten()  # Flatten for easier indexing


for index, year in enumerate(years):
    yearly_data = final_sorted[final_sorted['year'] == year]
    moran = Moran_Local(yearly_data['smr'], wq)
    ax = axs[index]
    show_legend = True if year == '2013' else False
    legend_kwds = {'fontsize': 8, 'title_fontsize': 'large', 'loc': 'upper right'}
    lisa_cluster(moran, yearly_data, p=0.05, ax=ax, legend=show_legend, legend_kwds=legend_kwds)
    ax.set_title(f"Year {year}")

plt.show()


In [ ]:
bins = [0, 5, 10, float('inf')]
labels = ['0-5', '6-10', '>10']
# Custom color map for different categories
cmap = ListedColormap(['#add8e6', '#ffa500', '#ff0000'])  # Light blue, orange, red

f, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(7.2, 2.8 * rows), constrained_layout=True)
axs = axs.flatten()  # Flatten for easier indexing

for index, year in enumerate(years):
    yearly_data = final_sorted[final_sorted['year'] == year].copy()
    # Categorize observed values
    yearly_data['category'] = pd.cut(yearly_data['observed'],include_lowest=True, bins=bins, labels=labels, right=True)
    
    # Plotting
    ax = axs[index]
    show_legend = True if year == '2013' else False
    yearly_data.plot(column='category', cmap=cmap, linewidth=0.5, ax=ax, edgecolor='black', legend=show_legend)
    ax.set_title(f"Year {year}")
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.set_xticks([])  # Remove x-axis ticks
    ax.set_yticks([])  # Remove y-axis ticks

# Hide any unused subplots in the grid
for idx in range(len(years), len(axs)):
    axs[idx].set_visible(False)
plt.show()

In [ ]:
bayes_map =gpd.read_file('data/map/merged_output.geojson')

In [ ]:
bayes_map.columns

In [ ]:
years = sorted(bayes_map['year'].unique())
num_years = len(years)
cols = 2
rows = (num_years + cols - 1) // cols  # Ceiling division

f, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(7.2, 2.8 * rows), constrained_layout=True)
axs = axs.flatten()  # Flatten for easier indexing


for index, year in enumerate(years):
    yearly_data = bayes_map[bayes_map['year'] == year]
    moran = Moran_Local(yearly_data['RR'], wq)
    ax = axs[index]
    show_legend = True if year == '2013' else False
    legend_kwds = {'fontsize': 8, 'title_fontsize': 'large', 'loc': 'upper right'}
    lisa_cluster(moran, yearly_data, p=0.05, ax=ax, legend=show_legend, legend_kwds=legend_kwds)
    ax.set_title(f"Year {year}")

plt.show()